In [1]:
using PlotlyJS
import IntervalRootFinding
import ForwardDiff
IRF = IntervalRootFinding;
FD = ForwardDiff


WebIO._IJuliaInit()

ForwardDiff

In [45]:
#Algunas Funciones Adicionales
direc(x) = x>0 ? "arrow-right" : "arrow-left"

function get_roots_and_stability(func;interval=[-10,10],nPoints=100)
    

    search_interval = IRF.Interval(interval[1], 
                                   interval[2])
    f_roots_intervals = IRF.roots(func,search_interval)
    f_roots = [IRF.mid(fr.interval) for fr in f_roots_intervals]
    f_roots = sort(f_roots)
    df_vals = FD.derivative.(func,f_roots)
    
    return (f_roots,df_vals)
end

function classify_pf(pf, df_vals_in_pf;eps =1e-7)
    atractors = pf[df_vals_in_pf.< -eps]
    repulsors = pf[df_vals_in_pf.> eps]
    marginals = pf[abs.(df_vals_in_pf).<eps]
    return (atractors,repulsors,marginals)
end


function d_fase(func; 
                interval=[-10,10],
                midpoints=nothing,
                nPoints=1000,eps=1e-7)
    " Calculo PF, la derivada para ver la estabilidad de los PF,
    evaluo en los puntos midpoints el flujo para ver hacia donde apunta
    "
    
    dx= (interval[2]-interval[1])/nPoints
    x = interval[1]:dx:interval[2]
    y = func.(x)
    y_range = [minimum(y),maximum(y)]
    
    
    f_roots,df_vals = get_roots_and_stability(func;interval,nPoints)
    
    atractors,repulsors,marginals = classify_pf(f_roots,df_vals;eps=eps) 
    #atractors = f_roots[df_vals.< -eps]
    #repulsors = f_roots[df_vals.> eps]
    #marginals = f_roots[abs.(df_vals).<eps]
    
    
    #si no se define inicialmente los puntos para evaluar el flujo
    if typeof(midpoints) ==Nothing
        points = cat(interval[1],f_roots,interval[2],dims=1)
        midpoints = [(p1+p2)/2 for (p1,p2) in zip(points[2:end],points[1:end-1])]
    end
    
    print( """La función tiene $(length(f_roots)) puntos fijos.
             Atractores: $atractors.
             Repulsores: $repulsors.
           """)
    #-----------------------
    #GRAFICOS
    #-----------------------
    #grafico de las flechas
    arrows = scatter(x = midpoints,
                     y = zeros(length(midpoints)),
                     mode="markers", marker_color="rgb(0,150,0)",
                     marker_symbol = direc.(func.(midpoints)),
                     marker_size = 15,
                     name="flujo")
    
    #grafico de los ejes

    x_ax = scatter(x=interval,y=[0,0],
                   line_color="rgb(125,125,125)",
                   name= "x_ax" ,
                   legendgroup="Axis",
                   legendgrouptitle_text="Axis")
    y_ax = scatter(x=[0,0], y=y_range,
                   line_color="rgb(125,125,125)",
                   name= "y_ax" ,
                   legendgroup="Axis")

    func_plot = scatter(x=x, y=y,
                        line_color= "rgb(0,0,0)" ,
                        name=L"$\frac{df}{dx}= f(x)$")
    
    atractors_plot = scatter(x=atractors, 
                             y=zeros(length(atractors)),
                             mode="markers",
                             marker_color="rgb(0,0,255)",
                             marker_size=10,
                             name="Atractores")
    
    repulsors_plot = scatter(x=repulsors,
                             y=zeros(length(repulsors)),
                             mode = "markers",
                             marker_color = "rgb(200,0,0)",
                             marker_size=10,
                             name="Repulsores")
    
    marginals_plot = scatter(x=marginals,
                             y=zeros(length(marginals)),
                             mode = "markers",
                             marker_color = "rgb(200,0,200)",
                             marker_size=10,
                             name="Marginales")
    #p2 = scatter(x=f_roots, y=repeat([0],length(f_roots)),
   #              mode="markers",
    #             marker_color=ar_color.(df_vals),
     #            marker_size=10)

    layout = Layout(xaxis_range=interval, 
                    yaxis_range=y_range)
    Plot([x_ax,y_ax,
            func_plot,
            atractors_plot,
            repulsors_plot,
            marginals_plot,
            arrows],layout)
    
end

function diag_bifurc(f; interval=[-10,10], 
                        nPoints = 100,     
                        par_vals = nothing,
                        eps=1e-7 )
    
    if typeof(par_vals)==Nothing
        par_vals = interval[1]:(interval[2]-interval[1])/nPoints:interval[2]
    end

    atr_list = [] #atractors
    rep_list = [] #repulsors
    mar_list = [] #marginals
    
    atr_x = [] #x_coord of atractors
    rep_x = [] #...
    mar_x = [] #...
    for a in par_vals
        this_f(x)=f(x,a,0)
        f_roots,df_vals=get_roots_and_stability(this_f)

        atractors,repulsors,marginals = classify_pf(f_roots,df_vals;eps=eps) 


        append!(atr_list, atractors)
        append!(rep_list, repulsors)
        append!(mar_list, marginals)
        append!(atr_x, fill(a, length(atractors)))
        append!(rep_x, fill(a, length(repulsors)))
        append!(mar_x, fill(a, length(marginals)))

    end
    
    max_y = maximum( cat(atr_list,
                         rep_list,
                         mar_list, dims=1))
    min_y = minimum( cat(atr_list,
                         rep_list,
                         mar_list, dims=1))
    min_y = min_y<0 ? min_y*1.1 : min_y*0.9
    max_y = max_y>0 ? max_y*1.1 : max_y*0.8
    
    y_range =[min_y,max_y]
    #------------------------------
    #plots
    #------------------------------
    repulsors_plot = scatter(x=rep_x,
                             y=rep_list,
                             mode = "markers",
                             marker_color = "rgb(200,0,0)",
                             marker_size=5,
                             name="Repulsores")
    atractors_plot = scatter(x=atr_x,
                             y=atr_list,
                             mode = "markers",
                             marker_color = "rgb(0,0,200)",
                             marker_size=5,
                             name="Atractores")
    marginals_plot = scatter(x=mar_x,
                             y=mar_list,
                             mode = "markers",
                             marker_color = "rgb(200,0,200)",
                             marker_size=5,
                             name="Marginales")
     

     x_ax = scatter(x=interval,y=[0,0],
                    line_color="rgb(125,125,125)",
                    name= "parámetro" ,
                    legendgroup="Axis",
                    legendgrouptitle_text="Axis")
     y_ax = scatter(x=[0,0], y=y_range,
                    line_color="rgb(125,125,125)",
                    name= "x" ,
                    legendgroup="Axis")
    
    layout = Layout(xaxis_range=interval, 
                     yaxis_range=y_range)
    Plot([x_ax, y_ax ,
          repulsors_plot,
          atractors_plot,
          marginals_plot],  layout)
end


diag_bifurc (generic function with 1 method)

Definimos las funciones

In [46]:
f_a(x,a,t) = a*x-x^3
f_b(x,a,t) = a*x+x^2/2 -x^3
f_c(x,a,t) = a/3-x^2+x^3/6
f_d(x,a,t) = a*x-x^3 +0.01
f_e(x,a,t) = a*x + x^3 - x^5

f_e (generic function with 1 method)

# Ejercicio 1
## Inciso a

In [18]:
#encuentro los ceros 

par_val=1
interval = [-2,2]

f_a_x(x)=f_a(x,par_val,0)
f_b_x(x)=f_b(x,par_val,0)
f_c_x(x)=f_c(x,par_val,0)
f_d_x(x)=f_d(x,par_val,0)
f_e_x(x)=f_e(x,par_val,0)


f_e_x (generic function with 1 method)

In [19]:
d_fase(f_a_x;interval=[-2,2])

La función tiene 3 puntos fijos.
  Atractores: [-1.0000000019151534, 1.0000000000287919].
  Repulsores: [1.9902240698034014e-9].


data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [20]:
d_fase(f_b_x;interval=[-2,2])

La función tiene 3 puntos fijos.
  Atractores: [-0.7807764064046888, 1.280776407435149].
  Repulsores: [1.6084589695874418e-19].


data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [21]:
d_fase(f_c_x;interval=[-2,8])

La función tiene 3 puntos fijos.
  Atractores: [0.6090936979685104].
  Repulsores: [-0.5524746105698758, 5.943380912688291].


data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [22]:
d_fase(f_d_x;interval=[-2,2])

La función tiene 3 puntos fijos.
  Atractores: [-0.9949619925459382, 1.0049629919522773].
  Repulsores: [-0.010001000300120032].


data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [23]:
d_fase(f_e_x;interval=[-2,2])

La función tiene 3 puntos fijos.
  Atractores: [-1.2720196495140428, 1.2720196505781538].
  Repulsores: [1.3529021991782499e-17].


data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [47]:
diag_bifurc(f_a;interval=[-2,2],nPoints = 1000)

data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [48]:
diag_bifurc(f_b;interval=[-2,2],nPoints=1000)

data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [54]:
diag_bifurc(f_c;interval=[-5,20],nPoints=1000)

data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [55]:
diag_bifurc(f_d;interval=[-2,2],nPoints=1000)

data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [58]:
diag_bifurc(f_e;interval=[-2,3],nPoints=1000)

data: [
  "scatter with fields legendgroup, legendgrouptitle, line, name, type, x, and y",
  "scatter with fields legendgroup, line, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"